# Pipeline

In [ ]:
# %pip install datasets
# # %pip uninstall transformers
# %pip install transformers
# %pip install nltk
# %pip install torch

In [1]:
task = "ner"

model_checkpoint = "ModelNerTceBr/ner-TceBr-Final-3-Model-2000lm-TceBr-bert-base-portuguese-cased/model/e5_lr0.0001"

In [2]:
import os
import io
import transformers
from transformers import pipeline
from transformers import AutoTokenizer
import pathlib
from pathlib import Path
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import json
import csv
import torch

print(transformers.__version__) # 4.34.1
print(torch.__version__) # 2.1.0+cu121

/home/sabrina/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.34.1
2.1.0+cu121


[nltk_data] Downloading package punkt to /home/sabrina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, padding='max_length', max_length=128, model_max_length=512, truncation=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, padding='max_length', max_length=128, truncation=True)

In [9]:
classificador = pipeline(task, model=model_checkpoint, tokenizer=tokenizer)

In [17]:
path_to_text_files = 'DadosTXT2'
path_to_save_results = 'DadosPipelineJson2/'
path_to_save_csv = 'DadosPipelineCsv2/'

p = Path(path_to_text_files).glob('**/*')
files = [x for x in p if x.is_file() and x.suffix == '.txt']
files

[PosixPath('DadosTXT2/009-673-2019-9_IP.txt'),
 PosixPath('DadosTXT2/033-538-2020-4_IP.txt')]

In [18]:
def sent_tokenize_with_word_limit(text, max_words):
    sentences = sent_tokenize(text)
    new_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) <= max_words:
            new_sentences.append(sentence)
        else:
            temp_sentence = []
            word_count = 0
            for word in words:
                if word_count + len(word.split()) <= max_words:
                    temp_sentence.append(word)
                    word_count += len(word.split())
                else:
                    new_sentences.append(' '.join(temp_sentence))
                    temp_sentence = [word]
                    word_count = len(word.split())
            if temp_sentence:
                new_sentences.append(' '.join(temp_sentence))
    return new_sentences

In [19]:
for file in files:
  try:
    resultados = list()
    with open(file, 'r') as f:
      data = f.read()
      max_words = 50
      sentences = sent_tokenize_with_word_limit(data, max_words)
      for sentence in sentences:
        if sentence != '':
          resultado = classificador(sentence)
          resultados.append(resultado)

    nome_arquivo_saida = path_to_save_results + file.stem + ".json"
    with open(nome_arquivo_saida, "w") as arquivo_saida:
      json.dump(str(resultados), arquivo_saida, indent=4)

    nome_arquivo_saida_csv = path_to_save_csv + file.stem + ".csv"

    campos = set()
    for lista in resultados:
        for objeto in lista:
            campos.update(objeto.keys())

    # Abra o arquivo CSV para escrita
    with open(nome_arquivo_saida_csv, mode='w', newline='', encoding='utf-8') as arquivo_csv:
      writer = csv.DictWriter(arquivo_csv, fieldnames=list(campos), delimiter=';')
      writer.writeheader()  # Escreve os cabeçalhos dos campos
      for lista in resultados:
          for objeto in lista:
              writer.writerow(objeto)  # Escreve cada objeto como uma linha no arquivo CSV

  except Exception as e:
    print("Problema com o arquivo: ", file)
    print("Sentence: ", sentence)
    print("Sentence Length: ", len(sentence))
    continue